# data preparation: 
unpack train and dev set and set expoleded version in apposite files

In [ ]:
import pandas as pd

# read the data from original datasets provided in the github repo for the task
df_arguments = pd.read_csv('./KPA_2021_shared_task/kpm_data/arguments_train.csv')[['arg_id', 'argument']]
df_key_points = pd.read_csv('./KPA_2021_shared_task/kpm_data/key_points_train.csv')[['key_point_id', 'key_point']]
df_labels = pd.read_csv('./KPA_2021_shared_task/kpm_data/labels_train.csv')

# arrays for data
labels = [None] * len(df_labels)  # labels
args = [None] * len(df_labels)  # arguments
kps = [None] * len(df_labels)  # key points

# fill the arrays. 
for i in range(len(df_labels)):
    label = df_labels.iloc[i]

    args[i] = df_arguments[df_arguments['arg_id'] == label['arg_id']]['argument'].array[0]
    kps[i] = df_key_points[df_key_points['key_point_id'] == label['key_point_id']]['key_point'].array[0]

# in this setting we want minimise the cosine similarity of encodings of related sentences,
# the desired otput will be values in the domain of the cosine-similarity function:
# cos. sim. = 1  if two vectors (encodings) are different,
#           = -1 if two vectors are similar
labels = list(map(lambda x: 0 if x == 0 else 1, df_labels['label']))

# now that we recontructed the full, preprocessed, training dataset, we save it in apposite files
args = pd.DataFrame(args)
args.to_csv('./args_train.csv')

kps = pd.DataFrame(kps)
kps.to_csv('./kps_train.csv')

labels = pd.DataFrame(labels)
labels.to_csv('./labels_train.csv')


In [ ]:

import pandas as pd
# read the data from original datasets provided in the github repo for the task
df_arguments = pd.read_csv('./KPA_2021_shared_task/kpm_data/arguments_dev.csv')[['arg_id', 'argument']]
df_key_points = pd.read_csv('./KPA_2021_shared_task/kpm_data/key_points_dev.csv')[['key_point_id', 'key_point']]
df_labels = pd.read_csv('./KPA_2021_shared_task/kpm_data/labels_dev.csv')

# arrays for data
labels = [None] * len(df_labels)  # labels
args = [None] * len(df_labels)  # arguments
kps = [None] * len(df_labels)  # key points

# fill the arrays. 
for i in range(len(df_labels)):
    label = df_labels.iloc[i]

    args[i] = df_arguments[df_arguments['arg_id'] == label['arg_id']]['argument'].array[0]
    kps[i] = df_key_points[df_key_points['key_point_id'] == label['key_point_id']]['key_point'].array[0]

# in this setting we want minimise the cosine similarity of encodings of related sentences,
# the desired otput will be values in the domain of the cosine-similarity function:
# cos. sim. = 1  if two vectors (encodings) are different,
#           = -1 if two vectors are similar
labels = list(map(lambda x: 0 if x == 0 else 1, df_labels['label']))
# now that we recontructed the full, preprocessed, training dataset, we save it in apposite files


args = pd.DataFrame(args)
args.to_csv('./args_dev.csv')

kps = pd.DataFrame(kps)
kps.to_csv('./kps_dev.csv')

labels = pd.DataFrame(labels)
labels.to_csv('./labels_dev.csv')


# start here
by setting gpus memory growth and re-reading previously prepared datasets

In [1]:
###### SET GPU: WHICH ONE TO USE AND MEMORY GROWTH ######
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    print("hello!")
    tf.config.set_visible_devices(gpus[1], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[1], True)

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# tf.keras.backend.set_floatx('float16')
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

hello!
4 Physical GPUs, 1 Logical GPUs


In [1]:
# re-read the training dataset from the relative files 
import pandas as pd 
kps_train = pd.read_csv('./kps_train.csv').to_numpy()[:,1]
args_train = pd.read_csv('./args_train.csv').to_numpy()[:,1]
labels_train = pd.read_csv('./labels_train.csv').to_numpy()[:,1]

kps_dev = pd.read_csv('./kps_dev.csv').to_numpy()[:,1]
args_dev = pd.read_csv('./args_dev.csv').to_numpy()[:,1]
labels_dev = pd.read_csv('./labels_dev.csv').to_numpy()[:,1]


In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
# per calcolare max len
# max(list(map(lambda x:len(x), args))) , max(list(map(lambda x:len(x), kps)))
MAX_LEN = 250
tokenized_args_train = tokenizer(args_train.tolist(), max_length=MAX_LEN, return_tensors='tf', padding='max_length')
tokenized_kps_train = tokenizer(kps_train.tolist(), max_length=MAX_LEN, return_tensors='tf', padding='max_length')

tokenized_args_dev = tokenizer(args_dev.tolist(), max_length=MAX_LEN, return_tensors='tf', padding='max_length')
tokenized_kps_dev = tokenizer(kps_dev.tolist(), max_length=MAX_LEN, return_tensors='tf', padding='max_length')

In [4]:
## TRAINING 
toks1_input_train = tokenized_args_train.input_ids
atts1_input_train = tokenized_args_train.attention_mask

toks2_input_train = tokenized_kps_train.input_ids
atts2_input_train = tokenized_kps_train.attention_mask

print('training data')
print(toks1_input_train.shape,atts1_input_train.shape,toks2_input_train.shape,atts2_input_train.shape)


## VALIDATION/DEV
toks1_input_dev = tokenized_args_dev.input_ids
atts1_input_dev = tokenized_args_dev.attention_mask

toks2_input_dev = tokenized_kps_dev.input_ids
atts2_input_dev = tokenized_kps_dev.attention_mask

print('validation/dev data')
print(toks1_input_dev.shape,atts1_input_dev.shape,toks2_input_dev.shape,atts2_input_dev.shape)


training data
(20635, 250) (20635, 250) (20635, 250) (20635, 250)
validation/dev data
(3458, 250) (3458, 250) (3458, 250) (3458, 250)


In [6]:
for i in range(10):
  print( tokenizer.decode(toks1_input_train[i])[0:100] ) 
  print( tokenizer.decode(toks2_input_train[i])[0:100] )
  print(labels_train[i])

[CLS] ` people reach their limit when it comes to their quality of life and should be able to end th
[CLS] assisted suicide gives dignity to the person that wants to commit it [SEP] [PAD] [PAD] [PAD] [
0
[CLS] a cure or treatment may be discovered shortly after having ended someone's life unnecessarily.
[CLS] assisted suicide allows people to solicit someone to die to their own benefit [SEP] [PAD] [PAD
0
[CLS] a cure or treatment may be discovered shortly after having ended someone's life unnecessarily.
[CLS] assisted suicide is akin to killing someone [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P
0
[CLS] a cure or treatment may be discovered shortly after having ended someone's life unnecessarily.
[CLS] assisted suicide should not be allowed because many times people can still get better [SEP] [P
1
[CLS] a cure or treatment may be discovered shortly after having ended someone's life unnecessarily.
[CLS] assisted suicide violates the doctor's role [SEP] [PAD] [PAD] [PAD] [PAD] [PA

In [7]:
for i in range(10):
  print( tokenizer.decode(toks1_input_dev[i])[0:100] ) 
  print( tokenizer.decode(toks2_input_dev[i])[0:100] )
  print(labels_dev[i])

[CLS] a real education is about giving students the tools to learn, think, and express themselves ; 
[CLS] school uniform is harming the student's self expression [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [P
1
[CLS] a real education is about giving students the tools to learn, think, and express themselves ; 
[CLS] school uniforms are expensive [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD
0
[CLS] a real education is about giving students the tools to learn, think, and express themselves ; 
[CLS] school uniforms are often uncomfortable / sexist [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA
0
[CLS] children express themselves through the clothes they wear and should be able to do this at sch
[CLS] school uniform harms learning / creativity [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA
0
[CLS] children express themselves through the clothes they wear and should be able to do this at sch
[CLS] school uniform is harming the student's self expression [SEP] [PAD] [PAD] [PA

In [8]:
import tensorflow as tf 

inputs_train = (
   toks1_input_train,
   atts1_input_train,
   toks2_input_train,
   atts2_input_train
  )
input_dataset_train = tf.data.Dataset.from_tensor_slices( inputs_train )
output_dataset_train = tf.data.Dataset.from_tensor_slices( labels_train )
dataset_train = tf.data.Dataset.zip( (input_dataset_train, output_dataset_train) )

inputs_dev = (
   toks1_input_dev,
   atts1_input_dev,
   toks2_input_dev,
   atts2_input_dev
  )
input_dataset_dev = tf.data.Dataset.from_tensor_slices( inputs_dev )
output_dataset_dev = tf.data.Dataset.from_tensor_slices( labels_dev )
dataset_dev = tf.data.Dataset.zip( (input_dataset_dev, output_dataset_dev) )


In [9]:
# from keras.layers import Lambda
from numpy import dtype
from transformers import TFBertModel

bert = TFBertModel.from_pretrained("bert-base-uncased", output_attentions=False, use_cache=False) #.bert
toks1=tf.keras.Input(shape=(MAX_LEN,), dtype='int32')
atts1=tf.keras.Input(shape=(MAX_LEN,), dtype="int32")
out1=bert(input_ids=toks1,attention_mask=atts1)

toks2=tf.keras.Input(shape=(MAX_LEN,), dtype='int32')
atts2=tf.keras.Input(shape=(MAX_LEN,), dtype="int32")
out2=bert(input_ids=toks2,attention_mask=atts2)

# prima provavamo a fare la media
# mean1=tf.reduce_mean(out1[0],1)
# mean2=tf.reduce_mean(out2[0],1)

# #########Comment this block if objective is cosine similarity calculation
# cosine_similarity=tf.keras.layers.Dot(axes=1,normalize=True)
# preds=cosine_similarity([mean1,mean2])

cls1 = out1[0][:,0,:]
cls2 = out2[0][:,0,:]

#########Comment this block if objective is cosine similarity calculation
cosine_similarity=tf.keras.layers.Dot(axes=1,normalize=True, dtype="float16")
preds=cosine_similarity([cls1,cls2])


model = tf.keras.Model(inputs=[toks1,atts1,toks2,atts2], outputs=preds)
# model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-05), metrics=['mse'])

print(model.summary())

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 250)]        0                                            
______________________________________________________________________________________________

In [15]:
import numpy as np

# max_queue_size=2
h = model.fit(dataset_train.batch(16), epochs = 1)

 204/1290 [===>..........................] - ETA: 17:15 - loss: 0.0725 - mse: 0.0725

In [ ]:
preds_train = model.predict(inputs_train)
pd.DataFrame(preds_train).to_csv('./preds_train.csv',index=False)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
preds_dev = model.predict(inputs_dev)
pd.DataFrame(preds_dev).to_csv('./preds_dev.csv',index=False)

In [ ]:
import pandas as pd
preds_dev = pd.read_csv('./preds_dev.csv')
labels = pd.read_csv('./KPA_2021_shared_task/kpm_data/labels_dev.csv')

print(len(preds_dev), len(labels))

jsons = {}
for i, line in preds_dev.T.iteritems():
  pred = line[0]
  line_labels = labels.iloc[i]
  arg_id = line_labels['arg_id']
  kp_id = line_labels['key_point_id']
  
  try:
    jsons[arg_id][kp_id] = pred
  except KeyError:
    jsons[arg_id] = {kp_id:pred}

import json
with open('preds_dev.json', 'w', encoding='utf-8') as f:
    json.dump(jsons, f, ensure_ascii=False, indent=4)

from KPA_2021_shared_task.code.track_1_kp_matching import load_kpm_data, get_predictions, evaluate_predictions

gold_data_dir = './KPA_2021_shared_task/kpm_data/'
predictions_file = './preds_dev.json'

arg_df, kp_df, labels_df = load_kpm_data(gold_data_dir, subset="dev")

merged_df = get_predictions(predictions_file, labels_df, arg_df, kp_df)
evaluate_predictions(merged_df)

3458 3458

ֿ** loading task data:
	(-1, 'We should abandon the use of school uniform'): loaded 121 arguments and 5 key points
	(-1, 'We should abolish the right to keep and bear arms'): loaded 123 arguments and 4 key points
	(-1, 'We should adopt an austerity regime'): loaded 108 arguments and 5 key points
	(-1, 'We should end affirmative action'): loaded 108 arguments and 3 key points
	(1, 'We should abandon the use of school uniform'): loaded 117 arguments and 5 key points
	(1, 'We should abolish the right to keep and bear arms'): loaded 110 arguments and 4 key points
	(1, 'We should adopt an austerity regime'): loaded 126 arguments and 5 key points
	(1, 'We should end affirmative action'): loaded 119 arguments and 5 key points

ֿ** loading predictions:
	loaded predictions for 932 arguments

** predictions analysis:
	(-1, 'We should abandon the use of school uniform'):
		submitted matched for 1.0 of the arguments (121/121)
	(-1, 'We should abolish the right to keep and bear arms'):
	

In [ ]:
import pandas as pd
preds_train = pd.read_csv('./preds_train.csv')
labels = pd.read_csv('./KPA_2021_shared_task/kpm_data/labels_train.csv')

print(len(preds_train), len(labels))

jsons = {}
for i, line in preds_train.T.iteritems():
  pred = line[0]
  line_labels = labels.iloc[i]
  arg_id = line_labels['arg_id']
  kp_id = line_labels['key_point_id']
  
  try:
    jsons[arg_id][kp_id] = pred
  except KeyError:
    jsons[arg_id] = {kp_id:pred}

import json
with open('preds_train.json', 'w', encoding='utf-8') as f:
    json.dump(jsons, f, ensure_ascii=False, indent=4)

from KPA_2021_shared_task.code.track_1_kp_matching import load_kpm_data, get_predictions, evaluate_predictions

gold_data_dir = './KPA_2021_shared_task/kpm_data/'
predictions_file = './preds_train.json'

arg_df, kp_df, labels_df = load_kpm_data(gold_data_dir, subset="train")

merged_df = get_predictions(predictions_file, labels_df, arg_df, kp_df)
evaluate_predictions(merged_df)

20635 20635

ֿ** loading task data:
	(-1, 'Assisted suicide should be a criminal offence'): loaded 121 arguments and 4 key points
	(-1, 'Homeschooling should be banned'): loaded 129 arguments and 6 key points
	(-1, 'The vow of celibacy should be abandoned'): loaded 112 arguments and 6 key points
	(-1, 'We should abandon marriage'): loaded 111 arguments and 5 key points
	(-1, 'We should abolish capital punishment'): loaded 110 arguments and 5 key points
	(-1, 'We should abolish intellectual property rights'): loaded 123 arguments and 4 key points
	(-1, 'We should adopt atheism'): loaded 123 arguments and 3 key points
	(-1, 'We should adopt libertarianism'): loaded 113 arguments and 5 key points
	(-1, 'We should ban human cloning'): loaded 123 arguments and 5 key points
	(-1, 'We should ban private military companies'): loaded 106 arguments and 4 key points
	(-1, 'We should ban the use of child actors'): loaded 121 arguments and 5 key points
	(-1, 'We should close Guantanamo Bay detentio

In [ ]:
for i,p in enumerate(preds_train):
  if i<100:
    print(labels_dev[i],p)
  else: 
    break

In [ ]:
# mean1=tf.reduce_mean(out1[0],1)
# mean2=tf.reduce_mean(out2[0],1)
cls1 = out1[0][:,0,:]
cls2 = out2[0][:,0,:]

#########Comment this block if objective is cosine similarity calculation
cosine_similarity=tf.keras.layers.Dot(axes=1,normalize=True)
preds=cosine_similarity([cls1,cls2])


model = Model(inputs=[toks1,atts1,toks2,atts2], outputs=preds)
# model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

print(model.summary())

In [ ]:
out1[0][:,0,:]

In [ ]:
import numpy as np
a = np.random.rand(768)
b = np.copy(a)
np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [ ]:
list(a)

In [ ]:
import numpy as np

######## BALANCE THE DATASET ########

# get a subset of the possible indices
rng = np.random.default_rng()
size = 15000
indeces1 = np.random.randint(len(toks1_input), size=size)
indeces2 = np.random.randint(len(toks1_input), size=size)

# get a random subset of the arguments
new_toks1 = np.take( toks1_input, indeces1 , axis=0)
new_atts1 = np.take( atts1_input, indeces1 , axis=0)

# get a random subset of the kps
new_toks2 = np.take( toks2_input, indeces2 , axis=0)
new_atts2 = np.take( atts2_input, indeces2 , axis=0)

# add the random subsets at the end 
toks1_input = np.vstack(( toks1_input, new_toks1))
atts1_input = np.vstack(( atts1_input, new_atts1))

toks2_input = np.vstack(( toks2_input, new_toks2))
atts2_input = np.vstack(( atts2_input, new_atts2))

# assume they don't match
labels = np.hstack((labels, np.array([-1]*size) ))

# shuffle the new dataset:

# take a vector [0,1,2,....,n]
order = np.arange(0,len(labels))

# shuffle it, so it becomes a random order
rng.shuffle(order)

# resave the elements in the random order
b_toks1_input = np.take(toks1_input,order, axis=0)
b_atts1_input = np.take(atts1_input,order, axis=0)

b_toks2_input = np.take(toks2_input,order, axis=0)
b_atts2_input = np.take(atts2_input,order, axis=0)

b_labels = np.take(labels,order, axis=0)


print(b_toks1_input.shape,b_atts1_input.shape,b_toks2_input.shape,b_atts2_input.shape, b_labels.shape)
sum(labels==1),           sum(labels==-1)